In [1]:
import copy
import math

# 外皮情報LV3からLV4へのコンバート

## Outline

- 外皮性能計算プログラムに、外皮の用途別情報を追加したものより、内壁等（建物内部の壁、天井、床）の面積の推定および熱貫流率の決定を行う。
- 外皮の用途別情報が入力された場合において床下空間に属する土間床等面積が0㎡より大きい場合、室と床下空間の間の床の面積の合計は、床下空間に属する土間床等面積の合計（ただし、室用途別の床面積と方位が「下面」の部位面積の差分の合計を上限とする）に等しいとみなす。室と床下空間の間の床は、室用途別の床面積と方位が「下面」の部位面積の差分に応じて、各用途に割り当てる。
- 室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。
- 内壁等の面積は、室の周長を求める際の係数（アスペクト比）と階高を仮定して、外皮の用途別情報および用途別床面積より推定する。主たる居室―非居室間、その他居室―非居室間は、それぞれ、各用途の床面積が0㎡より大きい場合は、居室と非居室は幅1m、高さ2m以上の通路等で繋がっていると想定する。
- 内壁等の熱貫流率は一般的な仕様を想定して与える（石膏ボード12.5mm－空気層－石膏ボード12.5mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。室内戸は考慮しない。
- その他は、内壁の用途別情報が得られている場合と同様に求める。

## 1. Common Element

#### Outline

'Common'要素はそのまま維持する。

参照：InputDataDefinition.ipynb

#### Function

In [2]:
def convert_common(d):
    return d['Common']

## 2. Walls Element

#### Outline

'Walls'要素はそのまま維持する。

#### Function

In [3]:
def convert_wall(d):
    return d['Walls']

## 3. Windows Element

#### Outline

'Windows'要素はそのまま維持する。

#### Function

In [4]:
def convert_window(d):
    return d['Windows']

## 4. Doors Element

#### Outline

'Doors'要素はそのまま維持する。

#### Function

In [5]:
def convert_door(d):
    return d['Doors']

## 5. Heatbridges Element

#### Outline

'Heatbridges'要素はそのまま維持する。

#### Function

In [6]:
def convert_heatbridge(d):
    return d['Heatbridges']

## 6. EarthfloorPerimeters Element

#### Outline

'EarthfloorPerimeters'要素はそのまま維持する。

#### Function

In [7]:
def convert_earthfloorperimeter(d):
    return d['EarthfloorPerimeters']

## 7. Earthfloors Element

#### Outline

'Earthfloors'要素はそのまま維持する。

#### Function

In [8]:
def convert_earthfloor(d):
    return d['EarthFloors']

## 8. InnerWalls Element

## 8.1. 用途別床面積を取得する関数

In [9]:
def get_area_floor(d):
    return d['Common']['MainOccupantRoomFloorArea'], \
           d['Common']['OtherOccupantRoomFloorArea'], \
           d['Common']['TotalFloorArea'] - d['Common']['MainOccupantRoomFloorArea'] - d['Common']['OtherOccupantRoomFloorArea']

## 8.2. InnerWalls Element - Ground Floor 

### 1) 層構成

#### Outline

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup> k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| 合板 | 0.012 | 0.16 | 0.075 | 720.0 |

#### Function

In [10]:
def get_inner_floor_layers():
    return [{'Name': 'PED', 'Cond': 0.16, 'Thick': 0.012, 'SpecH': 720.0}]

### 2) 床下空間上の床面積の推定

#### Outline

$$
\displaystyle
A_{f,MR}= A_f \times \frac{ A_{dif,MR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$$
\displaystyle
A_{f,OR}= A_f \times \frac{ A_{dif,OR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$$
\displaystyle
A_{f,NR}= A_f \times \frac{ A_{dif,NR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$A_{f,MR}$: 主たる居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{f,OR}$: その他の居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{f,NR}$: 非居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_f$: 室と床下空間の間の床面積の合計, m<sup>2</sup>  
$A_{dif,MR}$: 主たる居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  
$A_{dif,OR}$: その他の居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  
$A_{dif,NR}$: 非居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  

$$
\displaystyle
A_f= min(A_{dif,MR} + A_{dif,OR} + A_{dif,NR}, \sum_{i}^{}A_{b,i})
$$

$A_{b,i}$: 床下空間に属する土間床等の部位$i$の面積, m<sup>2</sup>

$$
\displaystyle
A_{dif, MR} = max(0, A_{MR} - \sum_i A_{of,MR,i} - \sum_i A_{ef,MR,i})
$$

$$
\displaystyle
A_{dif, OR} = max(0, A_{OR} - \sum_i A_{of,OR,i} - \sum_i A_{ef,OR,i})
$$

$$
\displaystyle
A_{dif, NR} = max(0, A_{NR} - \sum_i A_{of,NR,i} - \sum_i A_{ef,NR,i})
$$

$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{of,MR,i}$: 方位が「下面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,OR,i}$: 方位が「下面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,NR,i}$: 方位が「下面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{ef,MR,i}$: 主たる居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$: その他の居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$: 非居室の土間床$i$の面積, m<sup>2</sup>  

#### Function

In [11]:
def get_area_floor_over_underfloor(d):
    
    # 居室の方位が「下面」である外皮の部位と土間床等の面積の合計
    def get_A_of(d, s):
        # s : space name
        return sum( y['Area'] for y in d['Walls'] + d['Windows'] + d['Doors'] if y['Space'] == s and y['Direction'] == 'Bottom' ) \
                + sum( y['Area'] for y in d['EarthFloors'] if ('EarthFloors' in d ) == True and y['Space'] == s )
    
    # 床下空間に属する土間床等の面積の取得
    def get_A_b(d):
        return sum(earthfloor['Area'] for earthfloor in d['EarthFloors'] if earthfloor['Space'] == 'underfloor')
    
    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor(d)
    
    # 下面（direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    A_of_MR, A_of_OR, A_of_NR = get_A_of(d, 'Main'), get_A_of(d, 'Other'), get_A_of(d, 'Nonliving')
    
    # 方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差
    A_dif_MR, A_dif_OR, A_dif_NR = max(0.0, A_MR - A_of_MR), max(0.0, A_OR - A_of_OR), max(0.0, A_NR - A_of_NR)

    A_dif = A_dif_MR + A_dif_OR + A_dif_NR

    # 床下空間に属する土間床等の面積の取得
    A_b = get_A_b(d)

    #  室と床下空間の間の床面積の推定
    A_f = min(A_dif, A_b)

    # 居室と床下空間との間の床の面積
    A_f_MR, A_f_OR, A_f_NR = A_f * A_dif_MR / A_dif, A_f * A_dif_OR / A_dif, A_f * A_dif_NR / A_dif
   
    return {
        'Main' : A_f_MR,
        'Other' : A_f_OR,
        'Nonliving' : A_f_NR
    }

#### Example

In [12]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'Area': 60, 'Direction': 'Top', 'name': 'Ceiling', 'Space': 'Main', 'structure': 'wood', 'Type': 'Ceiling' },
        { 'Area': 15, 'Direction': 'Bottom', 'name': 'Floor', 'Space': 'Other', 'structure': 'wood', 'Type': 'Floor' },
    ],
    'Windows': [
        { 'Area': 30, 'Direction': 'SW', 'name': 'WindowSW', 'Space': 'Main', 'type': 'single' },
        { 'Area': 75, 'Direction': 'Bottom', 'name': 'WindowBottom', 'Space': 'Main', 'type': 'single' },
    ],
    'Doors': [
        { 'Area': 15, 'Direction': 'NW', 'name': 'DoorNW', 'Space': 'Nonliving' },
        { 'Area': 15, 'Direction': 'NE', 'name': 'DoorNE', 'Space': 'Other' }
    ],
    'EarthFloors': [
        { 'Area': 5, 'name': 'other', 'Space': 'Underfloor' },
        { 'Area': 5, 'name': 'entrance', 'Space': 'Underfloor' }
    ]
}

get_area_floor_over_underfloor(d)

{'Main': 0.0, 'Nonliving': 0.0, 'Other': 0.0}

### 3) 室と床下空間の間の床の構築

#### Function

In [13]:
def get_innerwall_groundfloor(d):
    
    # 室と床下空間の間の床面積の推定
    areas_innerfloor = get_area_floor_over_underfloor(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    ifs = []
    
    ifs.extend([ {
        'Name' : 'GroundFloor_' + space,
        'Type' : 'GroundFloor',
        'Direction' : 'Horizontal',
        'Area' : areas_innerfloor[space],
        'Layers' : get_inner_floor_layers(),
        'Spaces' : [space,'Underfloor'],
    } for space in ['Main', 'Other', 'Nonliving'] ])
        
    return ifs

## 8.3. InnerWalls Element - Inner Floor

### 1) 層構成

#### Outline

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup> k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| 合板 | 0.012 | 0.16 | 0.075 | 720.0 |

#### Function

In [14]:
def get_inner_floor_layers():
    return [{'Name': 'PED', 'Cond': 0.16, 'Thick': 0.012, 'SpecH': 720.0}]

### 2) 室上の床面積の推定

#### Outline

$$
\displaystyle
A_{if,MR}= \max \left(0, A_{MR} -  \left(A_{gf,MR} + \sum_i A_{of,MR,i} + \sum_i A_{ef,MR,i} \right)\right)
$$

$$
\displaystyle
A_{if,OR}= \max \left(0, A_{OR} -  \left(A_{gf,OR} + \sum_i A_{of,OR,i} + \sum_i A_{ef,OR,i} \right)\right)
$$

$$
\displaystyle
A_{if,NR}= \max \left(0, A_{NR} -  \left(A_{gf,NR} + \sum_i A_{of,NR,i} + \sum_i A_{ef,NR,i} \right)\right)
$$

$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{gf,MR}$: 主たる居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{gf,OR}$: その他の居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{gf,NR}$: 非居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{of,MR,i}$: 方位が「下面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,OR,i}$: 方位が「下面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,NR,i}$: 方位が「下面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{ef,MR,i}$: 主たる居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$: その他の居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$: 非居室の土間床$i$の面積, m<sup>2</sup>   

#### Function

In [15]:
def get_area_floor_over_room(d, innerwalls):
    
    # 床下空間上の床と、方位が「下面」である外皮の部位と、土間床等の面積の合計
    def get_A_f(d, s):
        # s : space name
        return sum( y['Area'] for y in innerwalls if (y['Type'] == 'GroundFloor'and y['Spaces'] == [s,'Underfloor'])) \
               + sum( y['Area'] for y in d['Walls'] + d['Windows'] + d['Doors'] if y['Space'] == s and y['Direction'] == 'Bottom' ) \
               + sum( y['Area'] for y in d['EarthFloors'] if ('EarthFloors' in d ) == True and y['Space'] == s )
    
    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor(d)
    
    # 床下空間上の床面積と下面（Direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    A_f_MR, A_f_OR, A_f_NR = get_A_f(d, 'Main'), get_A_f(d, 'Other'), get_A_f(d, 'Nonliving')
    
    # 床下空間上の床面積と方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差を、室上の床面積とみなす。
    A_if_MR, A_if_OR, A_if_NR = max(0.0, A_MR - A_f_MR), max(0.0, A_OR - A_f_OR), max(0.0, A_NR - A_f_NR)
    
    return {'Main': A_if_MR, 'Other': A_if_OR, 'Nonliving': A_if_NR}

#### Example

In [16]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'Area': 60, 'Direction': 'Top', 'name': 'Ceiling', 'Space': 'Main', 'structure': 'wood', 'type': 'Ceiling' },
        { 'Area': 15, 'Direction': 'Bottom', 'name': 'Floor', 'Space': 'Other', 'structure': 'wood', 'type': 'Floor' },
    ],
    'Windows': [
        { 'Area': 30, 'Direction': 'SW', 'name': 'WindowSW', 'Space': 'Main', 'type': 'single' },
        { 'Area': 75, 'Direction': 'Bottom', 'name': 'WindowBottom', 'Space': 'Main', 'type': 'single' },
    ],
    'Doors': [
        { 'Area': 15, 'Direction': 'NW', 'name': 'DoorNW', 'Space': 'Nonliving' },
        { 'Area': 15, 'Direction': 'NE', 'name': 'DoorNE', 'Space': 'Other' }
    ],
    'EarthFloors': [
        { 'Area': 5, 'name': 'other', 'Space': 'Underfloor' },
        { 'Area': 5, 'name': 'entrance', 'Space': 'Underfloor' }
    ]
}

innerwalls = [
        { 'Area': 60, 'Name': 'GroundFloor_Main', 'Type' : 'GroundFloor', 'Spaces': ['Main','Underfloor'], 'Direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'Area': 15, 'Name': 'GroundFloor_Other', 'Type' : 'GroundFloor', 'Spaces': ['Other','Underfloor'], 'Direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

get_area_floor_over_room(d, innerwalls)

{'Main': 0.0, 'Nonliving': 30.0, 'Other': 30.0}

### 3) 室下の天井面積の推定

#### Outline

$$
\displaystyle
A_{ic,MR}= \max \left(0, A_{MR} -  \sum_i A_{oc,MR,i} \right)
$$

$$
\displaystyle
A_{ic,OR}= \max \left(0, A_{OR} -  \sum_i A_{oc,OR,i} \right)
$$

$$
\displaystyle
A_{ic,NR}= \max \left(0, A_{NR} -  \sum_i A_{oc,NR,i} \right)
$$

$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{oc,MR,i}$: 方位が「上面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{oc,OR,i}$: 方位が「上面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{oc,NR,i}$: 方位が「上面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  

#### Function

In [17]:
def get_area_ceiling_below_room(d):
    
    # 方位が「上面」である外皮の部位の面積の合計
    def get_A_of(d, s):
        # s : space name
        return sum( y['Area'] for y in d['Walls'] + d['Windows'] + d['Doors'] if y['Space'] == s and y['Direction'] == 'Top' )
    
    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor(d)
    
    # 上面（Direction:Top）の壁・窓・ドア面積の合計値の取得
    A_oc_MR, A_oc_OR, A_oc_NR = get_A_of(d, 'Main'), get_A_of(d, 'Other'), get_A_of(d, 'Nonliving')
    
    # 方位が「上面」の部位の面積の合計値と、床面積の差
    A_ic_MR, A_ic_OR, A_ic_NR = max(0.0, A_MR - A_oc_MR), max(0.0, A_OR - A_oc_OR), max(0.0, A_NR - A_oc_NR)
   
    return {'Main': A_ic_MR, 'Other': A_ic_OR, 'Nonliving': A_ic_NR}

#### Example

In [18]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'Area': 60, 'Direction': 'Top', 'name': 'Ceiling', 'Space': 'Other', 'structure': 'wood', 'Type': 'Ceiling' },
        { 'Area': 15, 'Direction': 'Bottom', 'name': 'Floor', 'Space': 'Other', 'structure': 'wood', 'Type': 'Floor' },
    ],
    'Windows': [
        { 'Area': 30, 'Direction': 'SW', 'name': 'WindowSW', 'Space': 'Main', 'type': 'single' },
        { 'Area': 75, 'Direction': 'Bottom', 'name': 'WindowBottom', 'Space': 'Main', 'type': 'single' },
    ],
    'Doors': [
        { 'Area': 15, 'Direction': 'NW', 'name': 'DoorNW', 'Space': 'Nonliving' },
        { 'Area': 15, 'Direction': 'NE', 'name': 'DoorNE', 'Space': 'Other' }
    ],
    'EarthFloors': [
        { 'Area': 5, 'name': 'other', 'Space': 'Underfloor' },
        { 'Area': 5, 'name': 'entrance', 'Space': 'Underfloor' }
    ]
}

get_area_ceiling_below_room(d)

{'Main': 30.0, 'Nonliving': 30.0, 'Other': 0.0}

### 4) 室間の床と天井の面積の推定

#### Outline

$$
\displaystyle
A_{if,MR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,OR} = 0 ) \\
\frac{A_{ic,OR}}{A_{ic,OR}+A_{ic,NR}} \times A_{if,MR} & ( A_{ic,OR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{if,MR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,NR} = 0 ) \\
\frac{A_{ic,NR}}{A_{ic,OR}+A_{ic,NR}} \times A_{if,MR} & ( A_{ic,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{if,OR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,MR} = 0 ) \\
\frac{A_{ic,MR}}{A_{ic,MR}+A_{ic,NR}} \times A_{if,OR} & ( A_{ic,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{if,OR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,NR} = 0 ) \\
\frac{A_{ic,NR}}{A_{ic,MR}+A_{ic,NR}} \times A_{if,OR} & ( A_{ic,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{if,NR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,MR} = 0 ) \\
\frac{A_{ic,MR}}{A_{ic,MR}+A_{ic,OR}} \times A_{if,NR} & ( A_{ic,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{if,NR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,OR} = 0 ) \\
\frac{A_{ic,OR}}{A_{ic,MR}+A_{ic,OR}} \times A_{if,NR} & ( A_{ic,OR} > 0 ) \\
\end{array}
\right.
$$

$A_{if,MR,OR}$: その他の居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,MR,NR}$: 非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR,MR}$: 主たる居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,OR,NR}$: 非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR,MR}$: 主たる居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{if,NR,OR}$: その他の居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との、間の非居室の床の面積, m<sup>2</sup>  

$$
\displaystyle
A_{ic,MR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{if,OR} = 0 ) \\
\frac{A_{if,OR}}{A_{if,OR}+A_{if,NR}} \times A_{ic,MR} & ( A_{if,OR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,MR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{if,NR} = 0 ) \\
\frac{A_{if,NR}}{A_{if,OR}+A_{if,NR}} \times A_{ic,MR} & ( A_{if,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{ic,OR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{if,MR} = 0 ) \\
\frac{A_{if,MR}}{A_{if,MR}+A_{if,NR}} \times A_{ic,OR} & ( A_{if,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,OR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{if,NR} = 0 ) \\
\frac{A_{if,NR}}{A_{if,MR}+A_{if,NR}} \times A_{ic,OR} & ( A_{if,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{ic,NR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{if,MR} = 0 ) \\
\frac{A_{if,MR}}{A_{if,MR}+A_{if,OR}} \times A_{ic,NR} & ( A_{if,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,NR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{if,OR} = 0 ) \\
\frac{A_{if,OR}}{A_{if,MR}+A_{if,OR}} \times A_{ic,NR} & ( A_{if,OR} > 0 ) \\
\end{array}
\right.
$$

$A_{ic,MR,OR}$: その他の居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,MR,NR}$: 非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,MR}$: 主たる居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,NR}$: 非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,MR}$: 主たる居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,OR}$: その他の居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との、間の非居室の床の面積, m<sup>2</sup>  
$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  

#### Function

In [19]:
def get_area_slab_between_rooms(d, innerwalls):
    
    space = ['Main', 'Other', 'Nonliving']   
    # 室上の床面積の取得
    A_if =  get_area_floor_over_room(d, innerwalls)
    
    # 室下の天井面積の取得
    A_ic =  get_area_ceiling_below_room(d)

    # 室上の床面積を、他の用途の室下の天井面積で案分する。
    A_innerfloor = []
    for x in space :
        space1 = copy.deepcopy(space)
        space1.remove(x)
        for y in space1 :
            space2 = copy.deepcopy(space1)
            space2.remove(y)
            A_innerfloor.extend([
                {
                    'Name' : 'InnerFloor' + x + y,
                    'Area' : 0 if A_ic[y] == 0 else A_if[x] * A_ic[y] / (A_ic[y] + A_ic[space2[0]]),
                    'Space' : x,
                    'NextSpace' : y
                }
            ])

    # 室下の天井面積を、他の用途の室下上の床面積で案分する。
    A_innerceiling = []
    for x in space :
        space1 = copy.deepcopy(space)
        space1.remove(x)
        for y in space1 :
            space2 = copy.deepcopy(space1)
            space2.remove(y)
            A_innerceiling.extend([
                {
                    'Name' : 'InnerCeiling' + x + y,
                    'Area' : 0 if A_if[y] == 0 else A_ic[x] * A_if[y] / (A_if[y] + A_if[space2[0]]),
                    'Space' : x,
                    'NextSpace' : y
                }
            ])
  
    return {'InnerFloors' : A_innerfloor, 'InnerCeilings' : A_innerceiling}

#### Example

In [20]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'Area': 60, 'Direction': 'Top', 'name': 'Ceiling', 'Space': 'Other', 'structure': 'wood', 'Type': 'Ceiling' },
        { 'Area': 15, 'Direction': 'Bottom', 'name': 'Floor', 'Space': 'Other', 'structure': 'wood', 'Type': 'Floor' },
    ],
    'Windows': [
        { 'Area': 30, 'Direction': 'SW', 'name': 'WindowSW', 'Space': 'Main', 'type': 'single' },
        { 'Area': 75, 'Direction': 'Bottom', 'name': 'WindowBottom', 'Space': 'Main', 'type': 'single' },
    ],
    'Doors': [
        { 'Area': 15, 'Direction': 'NW', 'name': 'DoorNW', 'Space': 'Nonliving' },
        { 'Area': 15, 'Direction': 'NE', 'name': 'DoorNE', 'Space': 'Other' }
    ],
    'EarthFloors': [
        { 'Area': 5, 'name': 'other', 'Space': 'Underfloor' },
        { 'Area': 5, 'name': 'entrance', 'Space': 'Underfloor' }
    ]
}

innerwalls =  [
    { 'Area': 60, 'Name': 'GroundFloor_Main', 'Type' : 'GroundFloor', 'Spaces': ['Main','Underfloor'], 'Direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
    { 'Area': 15, 'Name': 'GroundFloor_Other', 'Type' : 'GroundFloor', 'Spaces': ['Other','Underfloor'], 'Direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

get_area_slab_between_rooms(d, innerwalls)

{'InnerCeilings': [{'Area': 15.0,
   'Name': 'InnerCeilingMainOther',
   'NextSpace': 'Other',
   'Space': 'Main'},
  {'Area': 15.0,
   'Name': 'InnerCeilingMainNonliving',
   'NextSpace': 'Nonliving',
   'Space': 'Main'},
  {'Area': 0,
   'Name': 'InnerCeilingOtherMain',
   'NextSpace': 'Main',
   'Space': 'Other'},
  {'Area': 0.0,
   'Name': 'InnerCeilingOtherNonliving',
   'NextSpace': 'Nonliving',
   'Space': 'Other'},
  {'Area': 0,
   'Name': 'InnerCeilingNonlivingMain',
   'NextSpace': 'Main',
   'Space': 'Nonliving'},
  {'Area': 30.0,
   'Name': 'InnerCeilingNonlivingOther',
   'NextSpace': 'Other',
   'Space': 'Nonliving'}],
 'InnerFloors': [{'Area': 0,
   'Name': 'InnerFloorMainOther',
   'NextSpace': 'Other',
   'Space': 'Main'},
  {'Area': 0.0,
   'Name': 'InnerFloorMainNonliving',
   'NextSpace': 'Nonliving',
   'Space': 'Main'},
  {'Area': 15.0,
   'Name': 'InnerFloorOtherMain',
   'NextSpace': 'Main',
   'Space': 'Other'},
  {'Area': 15.0,
   'Name': 'InnerFloorOtherNonli

### 5) 室と床下空間の間の床の構築

#### Function

In [21]:
def get_innerwall_groundfloor(d):
    
    # 室と床下空間の間の床面積の推定
    areas_groundfloor = get_area_floor_over_underfloor(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    ifs = []
    
    ifs.extend([ {
        'Name' : 'GroundFloor_' + space,
        'Type' : 'GroundFloor',
        'Direction' : 'Horizontal',
        'Area' : areas_groundfloor[space],
        'Layers' : get_inner_floor_layers(),
        'Spaces' : [space,'Underfloor'], 
    } for space in ['Main', 'Other', 'Nonliving'] ])
        
    return ifs

### 6) 室間の床・天井の構築

#### Function

In [22]:
def get_innerwall_innerslab(d, innerwalls):
    
    # 室間の床・天井面積の推定
    areas_innerslab = get_area_slab_between_rooms(d, innerwalls)
    print(areas_innerslab)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    ifs = []
    
    ifs.extend([ {
        'Name' : 'InnerFloor_' + x['Space'],
        'Type' : 'InnerFloor',
        'Direction' : 'Horizontal',
        'Area' : x['Area'],
        'Layers' : get_inner_floor_layers(),
        'Space' : x['Space'], 
        'NextSpace' : x['NextSpace'],
    } for x in areas_innerslab['InnerFloors'] ])
    
    ifs.extend([ {
        'Name' : 'InnerCeiling_' + x['Space'],
        'Type' : 'InnerCeiling',
        'Direction' : 'Horizontal',
        'Area' : x['Area'],
        'Layers' : get_inner_floor_layers(),
        'Space' : x['Space'], 
        'NextSpace' : x['NextSpace'],
    } for x in areas_innerslab['InnerCeilings'] ])
        
    return ifs

## 8.5. InnerWalls Element - Inner Wall 

### 1) 層構成

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup>2 k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |
| 非密閉空気層 | - | - | 0.090 | 0.0 |
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |

#### Function

In [23]:
def get_inner_wall_layers():
    return [
        {'Name': 'GPB', 'Cond': 0.22, 'Thick': 0.0125, 'SpecH': 830.0},
        {'Name': 'GPB', 'Cond': 1/0.09, 'Thick': 1, 'SpecH': 0},
        {'Name': 'GPB', 'Cond': 0.22, 'Thick': 0.0125, 'SpecH': 830.0}
    ]

### 3) 内壁等面積の推定

#### Outline

$$
A_{iw,MR-OR} = \left \{
\begin{array}{ll}
0.0 & ( A_{MR} = 0.0 \quad or \quad A_{OR} = 0.0 ) \\
\displaystyle
\max \left( 0.0, \frac{ A_{iw,MR} + A_{iw,OR} - A_{iw,NR} }{2} \right) & ( A_{MR} > 0.0 \quad and \quad A_{OR} > 0.0 )
\end{array}
\right.
$$

$$
A_{iw,MR-NR} = \left \{
\begin{array}{ll}
0.0 & ( A_{MR} = 0.0 \quad or \quad A_{NR} = 0.0 ) \\
\displaystyle
\max \left( 2.0, \frac{ A_{iw,MR} + A_{iw,NR} - A_{iw,OR} }{2} \right) & ( A_{MR} > 0.0 \quad and \quad A_{NR} > 0.0 )
\end{array}
\right.
$$

$$
A_{iw,NR-OR} = \left \{
\begin{array}{ll}
0.0 & ( A_{NR} = 0.0 \quad or \quad A_{OR} = 0.0 ) \\
\displaystyle
\max \left( 2.0, \frac{ A_{iw,NR} + A_{iw,OR} - A_{iw,MR} }{2} \right) & ( A_{NR} > 0.0 \quad and \quad A_{OR} > 0.0 )
\end{array}
\right.
$$

$ \because $

$$
A_{iw,MR} =A_{iw,MR-OR}+A_{iw,MR-NR}
$$

$$
A_{iw,OR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$$
A_{iw,NR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$A_{iw,MR-OR}$: 主たる居室とその他の居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR-NR}$: 主たる居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR-NR}$: その他の居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR}$: 主たる居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR}$: その他の居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,NR}$: 非居室の内壁等の面積, m<sup>2</sup>  

$$
A_{iw,MR} = \left \{
\begin{array}{ll}
0 & ( A_{MR} = 0 ) \\
A_{w,MR} - A_{ow,MR} = \max \left ( 0,4 \cdot ar_{MR} \cdot h \sqrt {A_{MR}} - A_{ow,MR} \right ) & ( A_{MR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,OR} = \left \{
\begin{array}{ll}
0 & ( A_{OR} = 0 ) \\
A_{w,OR} - A_{ow,OR} = \max \left ( 0, 4 \cdot ar_{NR} \cdot h \sqrt {A_{OR}} - A_{ow,OR} \right ) & ( A_{OR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,NR} = \left \{
\begin{array}{ll}
0 & ( A_{NR} = 0 ) \\
A_{w,NR} - A_{ow,NR} = \max \left ( 0, 4 \cdot ar_{OR} \cdot h \sqrt {A_{NR}} - A_{ow,NR} \right ) & ( A_{NR} > 0 )
\end{array}
\right.
$$

$A_{w,MR}$: 主たる居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{w,OR}$: その他の居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{w,NR}$: 非居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{ow,MR}$: 主たる居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$A_{ow,OR}$: その他の居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$A_{ow,NR}$: 非居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$ar_{MR}$: 主たる居室の周長を求める際の係数, -  
$ar_{OR}$: その他の居室の周長を求める際の係数, -  
$ar_{NR}$: 非居室の周長を求める際の係数, -  
$h$: 階高, m

#### Function

##### 内壁等の面積を推定する関数

In [24]:
def get_area_inner_wall(d):

    # 居室の外皮と床下空間との間の床の面積の合計を取得
    def get_A_ow(d):

        # 室用途別の内壁等の面積
        d_ow=[]
        for y in ['Walls', 'Windows', 'Doors']:
            if (y in d) == True :
                d_ow.extend(d[y])
        
        A_ow_MR = sum(y['Area'] for y in d_ow if (y['Space'] == 'Mmain' and y['Direction'] != 'Top' and y['Direction'] != 'Bottom'))
        A_ow_OR = sum(y['Area'] for y in d_ow if (y['Space'] == 'Other' and y['Direction'] != 'Top' and y['Direction'] != 'Bottom'))
        A_ow_NR = sum(y['Area'] for y in d_ow if (y['Space'] == 'Nonliving' and y['Direction'] != 'Top' and y['Direction'] != 'Bottom'))
        
        return A_ow_MR, A_ow_OR, A_ow_NR
    
    # 内壁等の面積の取得
    def get_A_iw(A_f, ar, A_ow):
        
        h = 2.8 # 階高, m
        
        return max(0.0, 4 * h * ar * A_f**0.5 - A_ow) if A_f > 0.0 else 0.0
        
    # 室用途間の内壁等の面積の推定
    # 室用途床面積が0㎡の場合に、室間の内壁等の面積を0㎡とする
    def get_A_iw_cor(A_s1, A_s2, A_iw_min, A_iw_s1, A_iw_s2, A_iw_os):
        if A_s1 == 0.0 or A_s2 == 0.0:
            return 0.0
        else:
            return max(A_iw_min, (A_iw_s1 + A_iw_s2 - A_iw_os)/2)

    # 周長を求める際の係数, -
    ar_MR, ar_OR, ar_NR = 1.2, 1.4, 1.4

    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor(d)
 
    # 居室の外皮と床下空間との間の床の面積の合計を取得
    A_ow_MR, A_ow_OR, A_ow_NR = get_A_ow(d)

    # 内壁等の面積の取得
    A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = get_A_iw(A_MR, ar_MR, A_ow_MR), get_A_iw(A_OR, ar_OR, A_ow_OR), get_A_iw(A_NR, ar_NR, A_ow_NR)

    # 室間の内壁等の面積の最小値, m2
    A_iw_min_MR_to_OR, A_iw_min_MR_to_NR, A_iw_min_OR_to_NR = 0.0, 2.0, 2.0
    
    
    # 室用途間の内壁等の面積の推定
    return {
        'main to other'      : get_A_iw_cor(A_MR, A_OR, A_iw_min_MR_to_OR, A_iw_space_MR, A_iw_space_OR, A_iw_space_NR),
        'main to nonliving'  : get_A_iw_cor(A_MR, A_NR, A_iw_min_MR_to_NR, A_iw_space_MR, A_iw_space_NR, A_iw_space_OR),
        'other to nonliving' : get_A_iw_cor(A_OR, A_NR, A_iw_min_OR_to_NR, A_iw_space_OR, A_iw_space_NR, A_iw_space_MR)
    }

#### Example

In [25]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 60.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'Area': 60, 'Direction': 'Top', 'name': 'Ceiling', 'Space': 'Main', 'structure': 'wood', 'Type': 'Ceiling' },
        { 'Area': 15, 'Direction': 'Bottom', 'name': 'Floor', 'Space': 'Other', 'structure': 'wood', 'Type': 'Floor' },
    ],
    'Windows': [
        { 'Area': 30, 'Direction': 'SW', 'name': 'WindowSW', 'Space': 'Main', 'type': 'single' },
        { 'Area': 75, 'Direction': 'Bottom', 'name': 'WindowBottom', 'Space': 'Other', 'type': 'single' },
    ],
    'Doors': [
        { 'Area': 15, 'Direction': 'NW', 'name': 'DoorNW', 'Space': 'Nonliving' },
        { 'Area': 15, 'Direction': 'NE', 'name': 'DoorNE', 'Space': 'Other' }
    ],
    'EarthFloors': [
        { 'Area': 40, 'name': 'other', 'Space': 'Underfloor' },
        { 'Area': 20, 'name': 'entrance', 'Space': 'Nonliving' }
    ]
}

get_area_inner_wall(d)

{'main to nonliving': 0.0,
 'main to other': 105.28127504155998,
 'other to nonliving': 0.0}

### 3) 内壁の構築

#### Function

In [26]:
def get_innerwall_wall(d):

    # 用途と複数用途間の対応関係の定義
    correspondence = {
        'main to other': ['Main', 'Other' ],
        'main to nonliving': ['Main', 'Nonliving' ],
        'other to nonliving': ['Other', 'Nonliving' ] 
    }  

    # 内壁面積の推定
    areas_innerwall = get_area_inner_wall(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    iws = []

    iws.extend([ {
        'Name' : 'Innerwall_' + c,
        'Type' : 'InnerWall',
        'Direction' : 'Vertical',
        'Layers' : get_inner_wall_layers(),
        'Area' : areas_innerwall[c],
        'Spaces' : correspondence[c]
    } for c in correspondence.keys()
    ])

    return iws

## 8.3. InnerWalls Element - 統合

#### Function

In [27]:
def convert_innerwall( d ):

    innerwalls =[]
    
    # 室と床下空間の間の床の推定
    innerwalls.extend(get_innerwall_groundfloor(d))
    
    # 室間の床・天井の推定
    innerwalls.extend(get_innerwall_innerslab(d, innerwalls))
    
    # 内壁等の推定              
    innerwalls.extend(get_innerwall_wall(d))
            
    return innerwalls

#### Example

In [28]:
d = {
    'Common': {'MainOccupantRoomFloorArea': 30.0, 'OtherOccupantRoomFloorArea': 60.0, 'Region': 6, 'TotalFloorArea': 120.0},
    'Doors': [
        {'Area': 0.63, 'Direction': 'NW', 'name': 'DoorNW_main', 'Space': 'Main'},
        {'Area': 1.26, 'Direction': 'NW', 'name': 'DoorNW_other', 'Space': 'Other'},
        {'Area': 0.63, 'Direction': 'NW', 'name': 'DoorNW_nonliving', 'Space': 'Nonliving'},
        {'Area': 0.54, 'Direction': 'NE', 'name': 'DoorNE_main', 'Space': 'Main'},
        {'Area': 1.08, 'Direction': 'NE', 'name': 'DoorNE_other', 'Space': 'Other'},
        {'Area': 0.54, 'Direction': 'NE', 'name': 'DoorNE_nonliving', 'Space': 'Nonliving'}
     ],
    'EarthfloorPerimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8, 'Space': 'Underfloor'},
        {'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8, 'Space': 'Underfloor'}
    ],
    'EarthFloors': [
        {'Area': 5.0, 'name': 'other', 'Space': 'Underfloor'},
        {'Area': 5.0, 'name': 'entrance', 'Space': 'Underfloor'}
    ],
    'Walls': [
        {'Area': 16.95, 'Direction': 'top', 'name': 'Ceiling_main', 'Space': 'Main', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 33.9, 'Direction': 'top', 'name': 'Ceiling_other', 'Space': 'Other', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 16.95, 'Direction': 'top', 'name': 'Ceiling_nonliving', 'Space': 'Nonliving', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 10.1575, 'Direction': 'SW', 'name': 'Wall_SW_main', 'Space': 'Main', 'structure': 'wood', 'Type': 'Wall'},
        {'Area': 20.315, 'Direction': 'SW', 'name': 'Wall_SW_other', 'Space': 'Other', 'structurre': 'wood', 'Type': 'Wall'},
        {'Area': 10.1575, 'Direction': 'SW', 'name': 'Wall_SW_nonliving', 'Space': 'Nonliving', 'structure': 'wood', 'Type': 'Wall'}
    ],
    'Windows': [
        {'Area': 7.5625, 'Direction': 'SW', 'name': 'WindowSW_main', 'Space': 'Main', 'type': 'single'},
        {'Area': 15.125, 'Direction': 'SW', 'name': 'WindowSW_other', 'Space': 'Other', 'type': 'single'},
        {'Area': 7.5625, 'Direction': 'SW', 'name': 'WindowSW_nonliving',  'Space': 'Nonliving', 'type': 'single'},
        {'Area': 0.7925, 'Direction': 'NW', 'name': 'WindowNW_main', 'Space': 'Main', 'type': 'single'},
        {'Area': 1.585, 'Direction': 'NW', 'name': 'WindowNW_other', 'Space': 'Other', 'type': 'single'},
        {'Area': 0.7925, 'Direction': 'NW', 'name': 'WindowNW_nonliving', 'Space': 'Nonliving', 'type': 'single'}
        ]
}

convert_innerwall( d )

{'InnerFloors': [{'Name': 'InnerFloorMainOther', 'Area': 20.0, 'Space': 'Main', 'NextSpace': 'Other'}, {'Name': 'InnerFloorMainNonliving', 'Area': 10.0, 'Space': 'Main', 'NextSpace': 'Nonliving'}, {'Name': 'InnerFloorOtherMain', 'Area': 30.0, 'Space': 'Other', 'NextSpace': 'Main'}, {'Name': 'InnerFloorOtherNonliving', 'Area': 30.0, 'Space': 'Other', 'NextSpace': 'Nonliving'}, {'Name': 'InnerFloorNonlivingMain', 'Area': 10.0, 'Space': 'Nonliving', 'NextSpace': 'Main'}, {'Name': 'InnerFloorNonlivingOther', 'Area': 20.0, 'Space': 'Nonliving', 'NextSpace': 'Other'}], 'InnerCeilings': [{'Name': 'InnerCeilingMainOther', 'Area': 20.0, 'Space': 'Main', 'NextSpace': 'Other'}, {'Name': 'InnerCeilingMainNonliving', 'Area': 10.0, 'Space': 'Main', 'NextSpace': 'Nonliving'}, {'Name': 'InnerCeilingOtherMain', 'Area': 30.0, 'Space': 'Other', 'NextSpace': 'Main'}, {'Name': 'InnerCeilingOtherNonliving', 'Area': 30.0, 'Space': 'Other', 'NextSpace': 'Nonliving'}, {'Name': 'InnerCeilingNonlivingMain', 'Are

[{'Area': 0.0,
  'Direction': 'Horizontal',
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}],
  'Name': 'GroundFloor_Main',
  'Spaces': ['Main', 'Underfloor'],
  'Type': 'GroundFloor'},
 {'Area': 0.0,
  'Direction': 'Horizontal',
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}],
  'Name': 'GroundFloor_Other',
  'Spaces': ['Other', 'Underfloor'],
  'Type': 'GroundFloor'},
 {'Area': 0.0,
  'Direction': 'Horizontal',
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}],
  'Name': 'GroundFloor_Nonliving',
  'Spaces': ['Nonliving', 'Underfloor'],
  'Type': 'GroundFloor'},
 {'Area': 20.0,
  'Direction': 'Horizontal',
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}],
  'Name': 'InnerFloor_Main',
  'NextSpace': 'Other',
  'Space': 'Main',
  'Type': 'InnerFloor'},
 {'Area': 10.0,
  'Direction': 'Horizontal',
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}],
  'Name'

## 3. Lv4入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床、8)内壁等の各コンバート関数を統合し、Lv3の入力情報をLv4の入力情報にコンバートする。

### Function

In [29]:
def convert(d):
    d_lv4_input = {}
    d_lv4_input['Common'] = convert_common(d)
    if ('Walls' in d) == True:
        d_lv4_input['Walls'] = convert_wall(d)
    if ('Windows' in d) == True:
        d_lv4_input['Windows'] = convert_window(d)
    if ('Doors' in d) == True:
        d_lv4_input['Doors'] = convert_door(d)
    if ('EarthfloorPerimeters' in d) == True:
        d_lv4_input['EarthfloorPerimeters'] = convert_earthfloorperimeter(d)
    if ('EarthFloors' in d) == True:
        d_lv4_input['EarthFloors'] = convert_earthfloor(d)
    if ('Heatbridges' in d) == True:
        d_lv4_input['Heatbridges'] = convert_heatbridge(d)
    d_lv4_input['InnerWalls'] = convert_innerwall(d)

    return d_lv4_input

### Example

In [30]:
d = {
    'Common': {'MainOccupantRoomFloorArea': 30.0, 'OtherOccupantRoomFloorArea': 60.0, 'Region': 6, 'TotalFloorArea': 120.0},
    'Doors': [
        {'Area': 0.63, 'Direction': 'NW', 'name': 'DoorNW_main', 'Space': 'Main'},
        {'Area': 1.26, 'Direction': 'NW', 'name': 'DoorNW_other', 'Space': 'Other'},
        {'Area': 0.63, 'Direction': 'NW', 'name': 'DoorNW_nonliving', 'Space': 'Nonliving'},
        {'Area': 0.54, 'Direction': 'NE', 'name': 'DoorNE_main', 'Space': 'Main'},
        {'Area': 1.08, 'Direction': 'NE', 'name': 'DoorNE_other', 'Space': 'Other'},
        {'Area': 0.54, 'Direction': 'NE', 'name': 'DoorNE_nonliving', 'Space': 'Nonliving'}
     ],
    'EarthfloorPerimeters': [
        {'Direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8, 'Space': 'Underfloor'},
        {'Direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8, 'Space': 'Underfloor'}
    ],
    'EarthFloors': [
        {'Area': 30.0, 'Name': 'Main', 'Space': 'Main'},
        {'Area': 5.0, 'Name': 'Entrance', 'Space': 'Nonliving'}
    ],
    'Walls': [
        {'Area': 16.95, 'Direction': 'top', 'name': 'Ceiling_main', 'Space': 'Main', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 33.9, 'Direction': 'top', 'name': 'Ceiling_other', 'Space': 'Other', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 16.95, 'Direction': 'top', 'name': 'Ceiling_nonliving', 'Space': 'Nonliving', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 10.1575, 'Direction': 'SW', 'name': 'Wall_SW_main', 'Space': 'Main', 'structure': 'wood', 'Type': 'Wall'},
        {'Area': 20.315, 'Direction': 'SW', 'name': 'Wall_SW_other', 'Space': 'Other', 'structure': 'wood', 'Type': 'Wall'},
        {'Area': 10.1575, 'Direction': 'SW', 'name': 'Wall_SW_nonliving', 'Space': 'Nonliving', 'structure': 'wood', 'Type': 'Wall'}
    ],
    'Windows': [
        {'Area': 7.5625, 'Direction': 'SW', 'name': 'WindowSW_main', 'Space': 'Main', 'type': 'single'},
        {'Area': 15.125, 'Direction': 'SW', 'name': 'WindowSW_other', 'Space': 'Other', 'type': 'single'},
        {'Area': 7.5625, 'Direction': 'SW', 'name': 'WindowSW_nonliving',  'Space': 'Nonliving', 'type': 'single'},
        {'Area': 0.7925, 'Direction': 'NW', 'name': 'WindowNW_main', 'Space': 'Main', 'type': 'single'},
        {'Area': 1.585, 'Direction': 'NW', 'name': 'WindowNW_other', 'Space': 'Other', 'type': 'single'},
        {'Area': 0.7925, 'Direction': 'NW', 'name': 'WindowNW_nonliving', 'Space': 'Nonliving', 'type': 'single'}
        ]
}

convert(d)

{'InnerFloors': [{'Name': 'InnerFloorMainOther', 'Area': 0.0, 'Space': 'Main', 'NextSpace': 'Other'}, {'Name': 'InnerFloorMainNonliving', 'Area': 0.0, 'Space': 'Main', 'NextSpace': 'Nonliving'}, {'Name': 'InnerFloorOtherMain', 'Area': 30.0, 'Space': 'Other', 'NextSpace': 'Main'}, {'Name': 'InnerFloorOtherNonliving', 'Area': 30.0, 'Space': 'Other', 'NextSpace': 'Nonliving'}, {'Name': 'InnerFloorNonlivingMain', 'Area': 10.0, 'Space': 'Nonliving', 'NextSpace': 'Main'}, {'Name': 'InnerFloorNonlivingOther', 'Area': 20.0, 'Space': 'Nonliving', 'NextSpace': 'Other'}], 'InnerCeilings': [{'Name': 'InnerCeilingMainOther', 'Area': 20.0, 'Space': 'Main', 'NextSpace': 'Other'}, {'Name': 'InnerCeilingMainNonliving', 'Area': 10.0, 'Space': 'Main', 'NextSpace': 'Nonliving'}, {'Name': 'InnerCeilingOtherMain', 'Area': 0, 'Space': 'Other', 'NextSpace': 'Main'}, {'Name': 'InnerCeilingOtherNonliving', 'Area': 60.0, 'Space': 'Other', 'NextSpace': 'Nonliving'}, {'Name': 'InnerCeilingNonlivingMain', 'Area': 0

{'Common': {'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'Doors': [{'Area': 0.63,
   'Direction': 'NW',
   'Space': 'Main',
   'name': 'DoorNW_main'},
  {'Area': 1.26, 'Direction': 'NW', 'Space': 'Other', 'name': 'DoorNW_other'},
  {'Area': 0.63,
   'Direction': 'NW',
   'Space': 'Nonliving',
   'name': 'DoorNW_nonliving'},
  {'Area': 0.54, 'Direction': 'NE', 'Space': 'Main', 'name': 'DoorNE_main'},
  {'Area': 1.08, 'Direction': 'NE', 'Space': 'Other', 'name': 'DoorNE_other'},
  {'Area': 0.54,
   'Direction': 'NE',
   'Space': 'Nonliving',
   'name': 'DoorNE_nonliving'}],
 'EarthFloors': [{'Area': 30.0, 'Space': 'Main', 'name': 'other'},
  {'Area': 5.0, 'Space': 'Underfloor', 'name': 'entrance'}],
 'EarthfloorPerimeters': [{'Direction': 'NW',
   'Space': 'Underfloor',
   'length': 2.43,
   'name': 'other_NW',
   'psi': 1.8},
  {'Direction': 'NE',
   'Space': 'Underfloor',
   'length': 2.43,
   'name': 'other_NE',
 